In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Sistemas de Recomendación

## Carga de Librerias

In [ ]:
!pip install unidecode

In [ ]:
nltk.download()

In [2]:
import numpy as np
import pandas as pd
import scipy
import sys
import re
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

sys.path.append(r"Utilities")
import Utilities 

## Lectura DataSet de Juguetes (usar encoding='iso-8859-1')

In [5]:
dataset = pd.read_csv('DatosJuguetes/datasetJuguetes.csv', delimiter=";",encoding='iso-8859-1')
dataset

,Url,Titulo,Descripcion,Tipo,Idioma
0,http://www.toysrus.es/product/index.jsp?produc...,Asalvo - Silla de Paseo Elba Rosa,Comprar Asalvo - Silla de Paseo Elba Rosa - La...,product,es_ES
1,http://www.toysrus.es/product/index.jsp?produc...,Zobo - Silla Paseo Moving Light,Comprar Zobo - Silla Paseo Moving Light - Zobo...,product,es_ES
2,http://www.toysrus.es/product/index.jsp?produc...,Maclaren - Silla Paseo Quest Medieval Azul/Plata,Comprar Maclaren - Silla Paseo Quest Medieval ...,product,es_ES
3,http://www.toysrus.es/product/index.jsp?produc...,Chicco - Silla Paseo London - Red Passion,Comprar Chicco - Silla Paseo London - Red Pass...,product,es_ES
4,http://www.toysrus.es/product/index.jsp?produc...,Chicco - Silla Paseo London - Matrix,Comprar Chicco - Silla Paseo London - Matrix -...,product,es_ES
...,...,...,...,...,...
5592,http://www.toysrus.es/product/index.jsp?produc...,Magic Box Huella,"Comprar Magic Box Huella - Magic Box Huella, u...",product,es_ES
5593,http://www.toysrus.es/product/index.jsp?produc...,Bright Starts - Mordedores Nuby,Comprar Bright Starts - Mordedores Nuby - Los ...,product,es_ES
5594,http://www.toysrus.es/product/index.jsp?produc...,Bright Stats - Mordedor Libro (varios modelos),Comprar Bright Stats - Mordedor Libro (varios ...,product,es_ES
5595,http://www.toysrus.es/product/index.jsp?produc...,Peppa Pig - Toalla de Playa,Comprar Peppa Pig - Toalla de Playa - Peppa Pi...,product,es_ES


## Breve Análsis descriptivo

Url            0
Titulo         0
Descripcion    0
Tipo           0
Idioma         0
dtype: int64

##### Conclusiones
- Hay 5 columnas de tipo string
- Se tiene un dataset de 5597x5
- No hay missing values

## Listado de opciones para recomendar (Listado juguetes)

0                      Asalvo - Silla de Paseo Elba Rosa
1                        Zobo - Silla Paseo Moving Light
2       Maclaren - Silla Paseo Quest Medieval Azul/Plata
3              Chicco - Silla Paseo London - Red Passion
4                   Chicco - Silla Paseo London - Matrix
                              ...                       
5592                                    Magic Box Huella
5593                     Bright Starts - Mordedores Nuby
5594      Bright Stats - Mordedor Libro (varios modelos)
5595                         Peppa Pig - Toalla de Playa
5596                         Star Wars - Toalla de Playa
Name: Titulo, Length: 5597, dtype: object

## Cálculo vectores descriptivos de Items

##### Limpieza de datos

In [8]:
#Tokenization


0    [comprar, asalvo, -, silla, paseo, elba, rosa,...
1    [comprar, zobo, -, silla, paseo, moving, light...
2    [comprar, maclaren, -, silla, paseo, quest, me...
3    [comprar, chicco, -, silla, paseo, london, -, ...
4    [comprar, chicco, -, silla, paseo, london, -, ...
5    [comprar, chicco, -, silla, paseo, echo, twin,...
6    [comprar, num, noms, -, pack, misterioso, (, v...
7    [comprar, princesas, disney, -, set, cenicient...
8    [comprar, masks, -, figura, gatuno, (, catboy,...
9    [comprar, you, &, -, pack, baberos, toalla, -,...
Name: Descripcion_tok, dtype: object

In [9]:
# Eliminar palabras con ausencia de significaddo (remover stopwords)
                                             

0    [comprar, asalvo, -, silla, paseo, elba, rosa,...
1    [comprar, zobo, -, silla, paseo, moving, light...
2    [comprar, maclaren, -, silla, paseo, quest, me...
3    [comprar, chicco, -, silla, paseo, london, -, ...
4    [comprar, chicco, -, silla, paseo, london, -, ...
5    [comprar, chicco, -, silla, paseo, echo, twin,...
6    [comprar, num, noms, -, pack, misterioso, (, v...
7    [comprar, princesas, disney, -, set, cenicient...
8    [comprar, masks, -, figura, gatuno, (, catboy,...
9    [comprar, you, &, -, pack, baberos, toalla, -,...
Name: Descripcion_tok, dtype: object

In [10]:
# Eliminar signos de puntuación (remover signos de puntuacion)


0    [comprar, asalvo, silla, paseo, elba, rosa, as...
1    [comprar, zobo, silla, paseo, moving, light, z...
2    [comprar, maclaren, silla, paseo, quest, medie...
3    [comprar, chicco, silla, paseo, london, red, p...
4    [comprar, chicco, silla, paseo, london, matrix...
5    [comprar, chicco, silla, paseo, echo, twin, ga...
6    [comprar, num, noms, pack, misterioso, varios,...
7    [comprar, princesas, disney, set, cenicienta, ...
8    [comprar, masks, figura, gatuno, catboy, luz, ...
9    [comprar, you, pack, baberos, toalla, you, pac...
Name: Descripcion_tok, dtype: object

In [11]:
# Union descripciones (pasar de array a string)


0    comprar asalvo silla paseo elba rosa asalvo si...
1    comprar zobo silla paseo moving light zobo sil...
2    comprar maclaren silla paseo quest medieval az...
3    comprar chicco silla paseo london red passion ...
4    comprar chicco silla paseo london matrix chicc...
5    comprar chicco silla paseo echo twin garnet ch...
6    comprar num noms pack misterioso varios modelo...
7    comprar princesas disney set cenicienta descub...
8    comprar masks figura gatuno catboy luz noche m...
9    comprar you pack baberos toalla you pack baber...
Name: Descripcion_clean, dtype: object

##### Cálculo Matriz Frecuencias (CountVectorizer(), usar encoding='iso-8859-1')

<5597x19148 sparse matrix of type '<class 'numpy.int64'>'
	with 214497 stored elements in Compressed Sparse Row format>

##### Visualización Matriz Frecuencias

,_ardtaloqko,aaa,aaron,abajo,abalanzarse,abalorio,abalorios,abandonada,abandonadas,abandonar,...,única,únicamente,únicas,único,únicos,úsala,úsela,útil,útiles,überjackal
Asalvo - Silla de Paseo Elba Rosa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zobo - Silla Paseo Moving Light,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Maclaren - Silla Paseo Quest Medieval Azul/Plata,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Chicco - Silla Paseo London - Red Passion,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Chicco - Silla Paseo London - Matrix,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Magic Box Huella,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bright Starts - Mordedores Nuby,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bright Stats - Mordedor Libro (varios modelos),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Peppa Pig - Toalla de Playa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Porcentaje de Ceros del total de elementos de la Matriz Frecuencias

99.79985603615951

##### Número de Ceros en cada fila

Mochila Pocoyo                              99.984333
Cars - DVD Cars                             99.984333
Papelera Spider-Man                         99.979110
Cubo Rubik's                                99.979110
Ultra - Pack 4 pilas 9V Ultra Alcalinas     99.973888
Ultra - Pack 6 Pilas C Ultra Alcalinas      99.973888
Ultra - Pack 6 Pilas D Ultra Alcalinas      99.973888
Ultra - Pack 20 Pilas AA Ultra Alcalinas    99.968665
Colchón Minicuna Espuma 90X40cm             99.968665
Duracell - 4 Pilas Duracell Ultra AA M3     99.968665
dtype: float64

##### Número de Ceros en cada columna

comprar        0.000000
años          33.893157
incluidas     79.131678
puede         81.615151
varios        81.793818
set           82.222619
colores       83.437556
cualquier     85.117027
accesorios    85.349294
puedes        86.349830
dtype: float64

##### Ejemplo de los valores diferentes de cero

acero          1
asalvo         2
años           1
capota         1
cesta          1
chasis         1
comprar        1
cubrepiés      1
dobles         1
elba           2
inferior       1
ligera         1
máxima         1
máximo         1
parte          1
paseo          2
peso           1
posiciones     1
reclinable     1
recomendado    1
respaldo       1
rosa           2
ruedas         1
silla          2
varias         1
ventana        1
Name: Asalvo - Silla de Paseo Elba Rosa, dtype: int64

##### Cálculo término frecuencia inversa de documento (TfidfTransformer())

https://mldeeplearning.com/text-vectorization-term-frequency-inverse-document-frequency-tfidf/

<5597x19148 sparse matrix of type '<class 'numpy.float64'>'
	with 214497 stored elements in Compressed Sparse Row format>

##### Visualizacion Matriz con términos frecuencia inversa de documento y normalizada

,_ardtaloqko,aaa,aaron,abajo,abalanzarse,abalorio,abalorios,abandonada,abandonadas,abandonar,...,única,únicamente,únicas,único,únicos,úsala,úsela,útil,útiles,überjackal
Asalvo - Silla de Paseo Elba Rosa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zobo - Silla Paseo Moving Light,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maclaren - Silla Paseo Quest Medieval Azul/Plata,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicco - Silla Paseo London - Red Passion,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicco - Silla Paseo London - Matrix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Magic Box Huella,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bright Starts - Mordedores Nuby,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bright Stats - Mordedor Libro (varios modelos),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Peppa Pig - Toalla de Playa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Ejemplo de los valores diferentes de cero

acero          0.149227
asalvo         0.385243
años           0.039712
capota         0.153682
cesta          0.157939
chasis         0.177924
comprar        0.028089
cubrepiés      0.205823
dobles         0.183676
elba           0.479283
inferior       0.170635
ligera         0.159514
máxima         0.148488
máximo         0.140867
parte          0.100637
paseo          0.246235
peso           0.122101
posiciones     0.126407
reclinable     0.144420
recomendado    0.198454
respaldo       0.145711
rosa           0.228831
ruedas         0.104549
silla          0.232182
varias         0.125750
ventana        0.168325
Name: Asalvo - Silla de Paseo Elba Rosa, dtype: float64

## Sistema Recomendación por Similitud entre items

Para cada objeto se calculan las similitud con el resto de objetos.

<br>
<br>

$$Similitud_{item} (x_1, x_2) = cos (\theta) = \frac {v_{x_1} \cdot v_{x_2}}{||v_{x_1}|| \cdot ||v_{x_2}||} = \frac {\sum_{i=1}^N (v_{x_1} * v_{x_2})}{\sum_{i=1}^N(v_{x_1,i}^2) * \sum_{i=1}^N(v_{x_2,i}^2)}$$

<br>

Para cada objeto se recomiendan los objetos con similitud más elevada.

<br>

##### Cálculo matriz similitudes

##### Validación de que la matriz anterior estaba normalizada (ver diagonal)

array([1., 1., 1., ..., 1., 1., 1.])

##### Visualizacion Matriz Similitudes

,Asalvo - Silla de Paseo Elba Rosa,Zobo - Silla Paseo Moving Light,Maclaren - Silla Paseo Quest Medieval Azul/Plata,Chicco - Silla Paseo London - Red Passion,Chicco - Silla Paseo London - Matrix,Chicco - Silla de Paseo Echo Twin - Garnet,Num Noms - Pack Misterioso (varios modelos),Princesas Disney - Set Cenicienta,PJ Masks - Figura de Gatuno (Catboy) con Luz,You & Me - Pack Baberos y Toalla,...,Mickey Mouse - Set de Cuna 2 Piezas Mickey,Soy Luna - Juego Sabanas 3 Piezas Soy Luna Love,Patrulla Canina - Juego Sábanas Rosa,Patrulla Canina - Juego Sábanas Leaders,Marco Foto y Doble Huella,Magic Box Huella,Bright Starts - Mordedores Nuby,Bright Stats - Mordedor Libro (varios modelos),Peppa Pig - Toalla de Playa,Star Wars - Toalla de Playa
Asalvo - Silla de Paseo Elba Rosa,1.000000,0.201578,0.188388,0.187367,0.189888,0.154296,0.001275,0.001712,0.002537,0.010156,...,0.000624,0.000707,0.035414,0.001325,0.000773,0.000813,0.000787,0.000612,0.000776,0.000809
Zobo - Silla Paseo Moving Light,0.201578,1.000000,0.221342,0.185328,0.187821,0.137246,0.000345,0.000463,0.000687,0.004958,...,0.000507,0.000574,0.000414,0.000359,0.000628,0.000660,0.007624,0.005929,0.000630,0.000657
Maclaren - Silla Paseo Quest Medieval Azul/Plata,0.188388,0.221342,1.000000,0.174866,0.177219,0.163420,0.000216,0.010058,0.000429,0.000326,...,0.027620,0.000359,0.000259,0.000224,0.000393,0.000413,0.000399,0.000311,0.000394,0.000411
Chicco - Silla Paseo London - Red Passion,0.187367,0.185328,0.174866,1.000000,0.889750,0.402563,0.000823,0.001105,0.001637,0.014069,...,0.000403,0.000457,0.000988,0.000855,0.000499,0.006809,0.000508,0.000395,0.000501,0.000522
Chicco - Silla Paseo London - Matrix,0.189888,0.187821,0.177219,0.889750,1.000000,0.407981,0.000834,0.001120,0.001659,0.014259,...,0.000408,0.000463,0.001001,0.000867,0.000506,0.006901,0.000515,0.000400,0.000508,0.000529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Magic Box Huella,0.000813,0.000660,0.000413,0.006809,0.006901,0.006317,0.000438,0.000588,0.000872,0.000661,...,0.000643,0.000729,0.000526,0.000455,0.267878,1.000000,0.000811,0.000631,0.000800,0.000834
Bright Starts - Mordedores Nuby,0.000787,0.007624,0.000399,0.000508,0.000515,0.000471,0.000424,0.000569,0.000844,0.009038,...,0.000623,0.006224,0.000509,0.000441,0.000771,0.000811,1.000000,0.163887,0.000774,0.007121
Bright Stats - Mordedor Libro (varios modelos),0.000612,0.005929,0.000311,0.000395,0.000400,0.016327,0.091156,0.000443,0.000656,0.013560,...,0.007374,0.004840,0.000396,0.002845,0.000600,0.000631,0.163887,1.000000,0.009167,0.005537
Peppa Pig - Toalla de Playa,0.000776,0.000630,0.000394,0.000501,0.000508,0.000465,0.000418,0.000562,0.000832,0.129019,...,0.021828,0.022465,0.016214,0.014032,0.000761,0.000800,0.000774,0.009167,1.000000,0.529172


##### Ejemplo Recomendación por Similitud entre items

Juguete: Soy Luna - Bolso Patines

 Los juguetes más similares son:


Soy Luna - Kit Decoración Patines                   0.873227
Color Me Mine - Soy Luna - Bolso Patines Sequeen    0.804420
Soy Luna - Patines Training T.30/31                 0.793340
Soy Luna - Patines Training T.32/33                 0.793340
Soy Luna - Patines Training T.34/35                 0.793340
Soy Luna - Patines Training T.36/37                 0.793340
Soy Luna - Patines Training T.38/39                 0.793340
Soy Luna - Puzzle 2 en 1                            0.774746
Soy Luna - Taza Microondas                          0.759405
Color Me Mine - Soy Luna - Mochila                  0.750709
Name: Soy Luna - Bolso Patines, dtype: float64

## Sistema Recomendación Filtrado Colaborativo Item-Item

Para cada usuario se propagan las valoraciones de cada objeto al resto mediante la métrica de similaridad entre los objetos

$$Score(u,x_i) = \sum\limits_{\substack{j=1 \\ j \neq i, \hspace{1mm} j \hspace{0.5mm} \epsilon \hspace{0.5mm} J}}^n w(rank_u(x_j)) * similitud_{item}(x_j, x_i)$$

Donde:
<ul>
    <li>$J$ es el conjunto de objetos valorados por el usuario $u$</li>
    <br>
    <li>$rank_u(x_j)$ es la valoración del usuario $u$ al objeto $x_j$</li>
    <br>
    <li>$w:[0-10] \to \mathbb{R}$ es una función que asigna a cada valoración un valor real</li>
</ul>

##### Simulación valoracion individual de un usuario

In [25]:
# Simulamos valoraciones del 1 al 10


# 90 % valoraciones iguales a 0 (NA's)


# Cálculos previos valoración real



# Valoración normalizada


array([0, 0, 0, ..., 0, 4, 0])

##### Ejemplo Recomendación de 10 juguetes por Filtrado Colaborativo Item-Item

In [26]:
# Pasamos a DataFrame las puntuaciones predichas para cada juguete

# Solo seleccionamos los juguetes que no han sido valorados por el usuario (Se supone que los valorados ya han sido comprados por el usuario)


# Ordenación decreciente

# Muestra el top 10 de los juguetes recomendados 


,Juguete
0,Peppa Pig - Poncho de Playa
1,Minnie Mouse - Toalla 70x140 cm + Bolsa (vario...
2,Buscando a Dory - Poncho de Playa
3,Minnie - Toalla 70x140 cm (varios modelos)
4,Nerf - Elite Strongarm
5,Mickey Mouse - Toalla 70x140 cm + Bolsa (vario...
6,Disney - Mochila de Playa Buscando a Dory
7,Mickey Mouse - Toalla 70x140 cm (varios modelos)
8,Buscando a Dory - Asiento Elevador Dory (De 15...
9,Peppa Pig - Figuras Peppa Pig y sus amigos (va...


## Sistema Recomendación Filtrado Colaborativo User-User

Permite estimar valoraciones de objetos que hayan valorado otros usuarios que tengan valoraciones en común con el usuario objetivo.

$$R(u_a, x) = \frac{\sum_{u_k \epsilon R_x} (z_{u_k, x} * similitud(u_a, u_k))}{\sum_{u_k \epsilon R_x} |similitud(u_a, u_k)|} * \sigma_a + \mu_a$$

$$similitud(u_a, u_k) = \frac{\sum_{i \epsilon L_{a,k}} (\upsilon_{k,i} - \mu_k)(\upsilon_{a,i} - \mu_a)}{\sqrt{\sum_{i \epsilon L_{a,k}} (\upsilon_{k,i} - \mu_k)^2} \sqrt{\sum_{i \epsilon L_{a,k}} (\upsilon_{a,i} - \mu_a)^2}}$$

$$z_{u_k, x} = \frac{\upsilon_{k, x} - \mu_k}{\sigma_k}$$

Donde:
<ul>
    <li>$u_a$ denota al usuario $a$</li>
    <li>$u_k$ denota al usuario $k$</li>
    <li>$x$ denota el objeto $x$ a valorar</li>
    <li>$R_x$ es el conjunto de usuarios que han valorado el objeto $x$</li>
    <li>$z_{u_k, x}$ es la normalización de la valoración del usuario $k$ sobre el objeto $x$</li>
    <li>$\sigma_a$ denota la desviación típica de las valoraciones del usuario $a$</li>
    <li>$\mu_a$ denota la media aritmética de las valoraciones del usuario $a$</li>
    <li>$\mu_k$ denota la media aritmética de las valoraciones del usuario $k$</li>
    <li>$L_{a,k}$ es el listado de objetos que han valorado tanto el usuario $a$ como el usuario $k$</li>
    <li>$\upsilon_{k,i}$ es la valoración media realizada por el usuario $k$ sobre el objeto $i$</li>
    <li>$\upsilon_{a,i}$ es la valoración media realizada por el usuario $a$ sobre el objeto $i$</li>    
</ul>

##### Lectura csv con valoraciones de 4 usuarios

,Titulo,User1,User2,User3,User4
0,Asalvo - Silla de Paseo Elba Rosa,NaN,NaN,NaN,NaN
1,Zobo - Silla Paseo Moving Light,NaN,NaN,NaN,NaN
2,Maclaren - Silla Paseo Quest Medieval Azul/Plata,NaN,NaN,NaN,NaN
3,Chicco - Silla Paseo London - Red Passion,NaN,NaN,NaN,NaN
4,Chicco - Silla Paseo London - Matrix,NaN,NaN,NaN,NaN
5,Chicco - Silla de Paseo Echo Twin - Garnet,5.0,NaN,7.0,1.0
6,Num Noms - Pack Misterioso (varios modelos),NaN,NaN,NaN,NaN
7,Princesas Disney - Set Cenicienta,NaN,NaN,NaN,NaN
8,PJ Masks - Figura de Gatuno (Catboy) con Luz,9.0,NaN,NaN,2.0
9,You & Me - Pack Baberos y Toalla,NaN,NaN,NaN,NaN


##### Ejemplo valoraciones user1

5     5.0
8     9.0
12    4.0
18    2.0
26    1.0
Name: User1, dtype: float64

##### Cálculo valoraciones normalizadas 

In [29]:
# Cálculos previos valoración real 


# Normalización valoraciones


,User1,User2,User3,User4
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
5592,0.0,0.0,0.0,0.0
5593,0.0,0.0,0.0,0.0
5594,0.0,0.0,0.0,0.0
5595,0.0,0.0,0.0,0.0


##### Cálculo Similitud entre usuarios

,User1,User2,User3,User4
User1,1.000000,-0.222837,0.325831,-0.635707
User2,-0.222837,1.000000,-0.406663,0.404145
User3,0.325831,-0.406663,1.000000,-0.176090
User4,-0.635707,0.404145,-0.176090,1.000000


##### Cálculo Scores Recomendación

array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       ...,
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]])

##### Ejemplo Recomendación juguetes por Filtrado Colaborativo User-User

In [32]:
RecomendacionFiltradoColaborativoUserUser = pd.DataFrame(data = Cociente, index = Listado.values)

# Ejemplo:
# Se selecciona usuario
Usuario = 4
# Se seleccionan aquellos elemento que no estan valorados por el usuario 4 y que tienen valores predichos por Filtrado Colaboratibo User-User

# Recomendación


,Juguete
0,Monster High - Maletín
1,Totally Me - Cuentas de Ensueño 3 en 1


## Sistema Recomendación Reducción Dimensionalidad 


Surprise es una librería específicamente diseñada para construir sistemas de recomendación en Python. Puede considerarse como la versión de scikit-learn para algoritmos de recomendación. `surprise` se ha convertido en una de las librerías para algoritmos de recomendacón más populares en python. Esto se debe a que es extremadamente robusto y fácil de usar.

La gran ventaja es que nos ofrece implementaciones de los algoritmos de filtro colaborativo más populares. Además también nos permitiria incorporar nuestro propio algoritomo dentro de su framework.

<center><img src='https://image.slidesharecdn.com/4772391/95/building-a-recommendation-engine-an-example-of-a-product-recommendation-engine-21-728.jpg' width=600 ></center>

In [33]:
#!pip install surprise

In [34]:
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import SVD

# Un reader es todavia necesario pero solo parametro rating_scale es requerido

### code ###

df = pd.read_csv("DatosJuguetes/u.data", delimiter="\t", header = None, encoding='iso-8859-1')
df.columns= ['userID', 'itemID', 'rating', 'timestamp']
# Las columnas deben corresponder a user id, item id and ratings (en este orden).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

# Metodo seleccionado


In [41]:
df

,userID,itemID,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [39]:
# train and tets


# Entrenamiento y predicción


Prediction(uid=684, iid=376, r_ui=3.0, est=2.897256111796258, details={'was_impossible': False})

In [38]:
# Accuracy RMSE


RMSE: 0.9398


0.9398384830332998

## Más algoritmos en el paquete `surprise`

No vamos a entrar en detalles del resto de algoritmos que contiene el paquete, pero vamos a listarlos para que los conozcaís y valoreis su utilización en el futuro.




### NormalPredictor

NormalPredictor predice los ratings de forma aleatoria entendiendo que la distribución de las puntuaciones sigue una normal. Es uno de los algoritmos más básicos que podemos encontrar.




### BaselineOnly
BaselineOnly nos devolverá el baseline para la combinación de usuario e ítem.




### k-NN algorithms

#### KNNBasic
KNNBasic es un algoritmo básico de filtro colaborativo.



#### KNNWithMeans
KNNWithMeans es un algoritmo de filtro colaborativo, que tienen en cuenta la media de los ratings de cada usuario. 



#### KNNWithZScore
KNNWithZScore es un algoritmo de filtro colaborativo, que tienen en cuenta la normalización z-score de las puntuaciones de cada usuario.



#### KNNBaseline
KNNBaseline es un algoritmo de filtro colaborativo, basado en el baseline. 


### Matrix Factorization-based algorithms


#### SVD y SVDpp

SVD y SVDpp son dos versiones del algoritmo que hemos analizado anteriormente.



#### NMF

NMF  es un algoritmo de filtro colaborativo basado en Non-negative Matrix Factorization. Conceptualmente es muy similar a  SVD.


### Slope One
SlopeOne es una implementación del algoritmo SlopeOne. https://en.wikipedia.org/wiki/Slope_One



### Co-clustering

Algortimo de filtro colaborativo basado en co-clustering.